In [ ]:
# 導入所需套件
import pandas as pd                 # 數據處理和分析
import numpy as np                  # 數值運算
import matplotlib.pyplot as plt     # 基礎繪圖
import seaborn as sns               # 進階繪圖，基於 matplotlib
from scipy import stats             # 用於統計檢定
import warnings                     # 用於處理警告訊息